# Non-stationary Bandit Problem

In [ ]:
def incremental_update(Q, Times,action, reward):
    """
    Update the action-value estimate Q for the given action and reward using an incremental update rule.

    Parameters:
    Q (list): A list of action-value estimates for each action.
    Times (list): A list of counts of how many times each action has been taken.
    action (int): The index of the action taken.
    reward (float): The reward received after taking the action.

    Returns:
    list: Updated list of action-value estimates.
    """
    
    Q[action] = Q[action] + (1.0 / Times[action]) * (reward - Q[action])

    return Q


In [ ]:
import numpy as np

# We don't want to use the argmax function from numpy because it doesn't break ties randomly. 
# We want to implement our own version of argmax that breaks ties randomly.

def argmax(q_values):
    """
    Takes in a list of q_values and returns the index of the item 
    with the highest value. Breaks ties randomly.
    returns: int - the index of the highest value in q_values
    """
    top_value = float("-inf")
    ties = []
    
    for i in range(len(q_values)):
        # if a value in q_values is greater than the highest value update top and reset ties to zero
        # if a value is equal to top value add the index to ties
        
        if q_values[i] > top_value:
            ties = []
            top_value = q_values[i]
            ties.append(i)
        elif q_values[i] == top_value:
            top_value = q_values[i]
            ties.append(i)
        
    # return a random selection from ties.
    return np.random.choice(ties) 


def greedy_action_selection(Q):
    """
    Select an action using a greedy action selection strategy based on the action-value estimates Q.

    Parameters:
    Q (list): A list of action-value estimates for each action.

    Returns:
    int: The index of the selected action.
    """
    return argmax(Q) 

def epsilon_greedy_action_selection(Q, epsilon):
    """
    Select an action using an epsilon-greedy action selection strategy based on the action-value estimates Q.

    Parameters:
    Q (list): A list of action-value estimates for each action.
    epsilon (float): The probability of selecting a random action (exploration rate).

    Returns:
    int: The index of the selected action.
    """
    if np.random.rand() < epsilon:
        return np.random.randint(len(Q))  # Explore: select a random action
    else:
        return argmax(Q)  # Exploit: select the action with the highest value

## Using an epsilon-greedy policy to solve a non-stationary bandit problem

The code bellow implements a simple epsilon-greedy algorithm to solve a non-stationary bandit problem. The environment is created using the library `buffalo_gym` and the environment is `Buffalo-v0` with `dynamic_rate=100`. The environment has 10 arms and the rewards are generated from a normal distribution. The rewards are non-stationary, meaning that the mean of the rewards changes over time. The algorithm will be tested for 1000 steps and the average reward will be plotted over time.

In [ ]:
# In this version, we will run 1000 steps of the 
# bandit problem and calculate the accumulated reward at each step.
# We will run 100 times and average the rewards over time to see if 
# the agent is learning to select the best action.

import gymnasium as gym
import buffalo_gym

arms = 10
steps = 1000
runs = 2000

average_rewards = [0.0 for _ in range(steps)]

for run in range(runs):

    Q = [0.0 for _ in range(arms)]
    Times = [0 for _ in range(arms)]

    Rewards = [0.0 for _ in range(steps)]
    env = gym.make("Buffalo-v0", arms=arms, dynamic_rate=100)

    obs = env.reset()
    step = 0
    while step < steps:
        action = epsilon_greedy_action_selection(Q, epsilon=0.1)
        #action = greedy_action_selection(Q)
        obs, reward, terminated, truncated, info = env.step(action)
        #print(f"Action: {action} - Reward: {reward}")
        Times[action] += 1
        Q = incremental_update(Q, Times, action, reward)
        Rewards[step] = reward
        step += 1

    env.close()
    #print("Final action-value estimates:", Q)
    average_rewards += np.array(Rewards)

average_rewards /= runs

# plot the average rewards over steps
import matplotlib.pyplot as plt
plt.plot(average_rewards)
plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.title("Average Reward over Steps")
plt.show()

## Questions

* What happened to the average reward over time? Did it converge to a stable solution or did it keep fluctuating? Why?

## Change the way how Q values are updated 

Use a constant step size alpha to update the Q values instead of using the sample average method.

In [ ]:
def incremental_update(Q, alpha, action, reward):
    """
    Update the action-value estimate Q for the given action and reward using an incremental update rule.

    Parameters:
    Q (list): A list of action-value estimates for each action.
    alpha (float): The step size parameter.
    action (int): The index of the action taken.
    reward (float): The reward received after taking the action.

    Returns:
    list: Updated list of action-value estimates.
    """
    
    Q[action] = Q[action] + alpha * (reward - Q[action])

    return Q

In [ ]:
import gymnasium as gym
import buffalo_gym

arms = 10
steps = 1000
runs = 2000

average_rewards_a = [0.0 for _ in range(steps)]

for run in range(runs):

    Q = [0.0 for _ in range(arms)]
    Times = [0 for _ in range(arms)]

    Rewards = [0.0 for _ in range(steps)]
    env = gym.make("Buffalo-v0", arms=arms, dynamic_rate=100)

    obs = env.reset()
    step = 0
    while step < steps:
        action = epsilon_greedy_action_selection(Q, epsilon=0.1)
        #action = greedy_action_selection(Q)
        obs, reward, terminated, truncated, info = env.step(action)
        #print(f"Action: {action} - Reward: {reward}")
        Times[action] += 1
        Q = incremental_update(Q, alpha=0.01, action=action, reward=reward)
        Rewards[step] = reward
        step += 1

    env.close()
    #print("Final action-value estimates:", Q)
    average_rewards_a += np.array(Rewards)

average_rewards_a /= runs

# plot the average rewards over steps
import matplotlib.pyplot as plt
plt.plot(average_rewards)
plt.plot(average_rewards_a)
plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.title("Average Reward over Steps")
plt.legend(["Sample Average", "Constant Step Size"])
plt.show()

## Activity

* Select different values of alpha (e.g., 0.1, 0.4, 0.9) in the code above and observe how the average reward changes over time.

## Questions

* Which method performs better in this non-stationary environment? Why?
* Which value of alpha performs better in this non-stationary environment? Why?
* How does the choice of alpha affect the performance of the algorithm? What happens if alpha is too high or too low?
* How does the performance of the algorithm change over time? Does it converge to a stable solution or does it keep fluctuating? Why?